The intention of this notebook is to ***sample/create the sequence data*** from the  T251 dataset provided by Merck&Co. 

##Notebook Setup##

In [0]:
#Imports:
import os
import numpy as np
import pandas as pd
import time
import requests
import json
from collections import defaultdict

In [0]:
#Connect to google drive:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Set up file paths:
data_folder_loc = "gdrive/My Drive/iGEM/Databases/Merck&Co/Data/Source/"
merck_and_co_excel_loc = data_folder_loc + "merck_and_co.xls"
target_data_folder_loc = "gdrive/My Drive/iGEM/Databases/Merck&Co/Data/T251/sequence_sampler_T251/"
sequence_dict_loc = target_data_folder_loc+"sequence_dict_T251.json"
print(os.path.isdir(data_folder_loc))
print(os.path.isdir(target_data_folder_loc))
print(os.path.isfile(merck_and_co_excel_loc))

True
True
True


## Sample Base Data ##

In [0]:
#Read excel table T1626:
T251_table = pd.read_excel(io=merck_and_co_excel_loc, sheet_name=1, header=0)

In [0]:
T251_table.head()

,MUTATION,DTM,TOTALAREA,HYDROPHOBICAREA,HYDROPHOBICRATIO,FRACTIONBURIED,BETA_FIRST,BETA_SECOND,BETA_DIFF,ALPHA_FIRST,...,R_BIOLUMINATE_DELTA_STABILITY_SOLV_SA,R_BIOLUMINATE_DELTA_STABILITY_SELFCONT,R_BIOLUMINATE_DELTA_STABILITY_PACKING,R_BIOLUMINATE_DELTA_STABILITY_COVALENT,R_BIOLUMINATE_DELTA_STABILITY_REFERENCE,R_BIOLUMINATE_DELTA_STABILITY_VDW,R_BIOLUMINATE_DELTA_STABILITY_HBOND,R_BIOLUMINATE_DELTA_STABILITY_COULOMB,R_BIOLUMINATE_DELTA_STABILITY_SOLV_GB,R_BIOLUMINATE_DELTA_STABILITY_LIPO
0,1ADO@A@D128A,-11.9,189.689,23.632,0.124583,0.497273,0.55,0.75,0.20,0.89,...,0,0.000000,0.000000,5.286400,-36.41,0.235717,3.542588,59.024591,-22.998830,-0.029323
1,1ADO@A@D128G,-12.7,189.689,23.632,0.124583,0.497273,0.55,0.65,0.10,0.89,...,0,0.000000,0.000000,1.797503,-32.57,0.680448,3.545229,59.574440,-23.925944,0.897229
2,1ADO@A@D128N,-11.7,189.689,23.632,0.124583,0.497273,0.55,0.63,0.08,0.89,...,0,-4.167533,0.000000,3.228514,5.31,-1.483240,2.825580,20.603743,-20.111146,0.030040
3,1ADO@A@D128Q,-11.6,189.689,23.632,0.124583,0.497273,0.55,0.76,0.21,0.89,...,0,-0.086061,-0.173941,8.269074,2.90,-2.150919,3.266579,8.343113,-17.284814,-0.811103
4,1ADO@A@D128V,-12.2,189.689,23.632,0.124583,0.497273,0.55,1.86,1.31,0.89,...,0,0.000000,0.000000,6.895204,-44.23,0.288068,3.544219,63.962126,-21.965650,-1.123189


## Sample sequences

### Retrieve IDs

In [0]:
pdb_ids = T251_table["MUTATION"].apply(lambda x: x.split("@")[0]).unique()

In [0]:
pdb_ids



```
array(['1ADO', '1ANT', '1ARR', '1B5M', '1BNI', '1BPI', '1BTM', '1C5G',
       '1CLW', '1CTS', '1EVQ', '1FLV', '1FTG', '1GUY', '1H0C', '1HFZ',
       '1HUE', '1HYN', '1I5T', '1IOJ', '1KA6', '1KDX', '1LS4', '1N0J',
       '1OSI', '1PIN', '1RGG', '1RN1', '1ROP', '1RTB', '1STN', '1TFE',
       '1TPE', '1WQ5', '1YPI', '2AFG', '2CBR', '2CI2', '2CRK', '2DRI',
       '2LZM', '3SSI', '5CRO'], dtype=object)
```



In [0]:
def fasta_to_dict(fasta_string):
  result_dict = {}
  fasta_split = fasta_string.replace("\n","").replace(" ","").split(">")[1:]
  for split in fasta_split:
    temp = split.split("|")
    result_dict[temp[0].split(":")[1]] = temp[-1][8:]
  return result_dict

In [0]:
def request_pdb_id_sequence(pdb_id):
  url = 'https://www.rcsb.org/pdb/download/downloadFastaFiles.do?'
  params = {
    'structureIdList':pdb_id,
    'compressionType':'uncompressed'
  }
  r = requests.post(url, data = params, timeout=60)
  temp_result = fasta_to_dict(r.text)
  return {pdb_id:temp_result}

In [0]:
def create_pdb_sequence_dict_requester(list_of_ids, destination_save):
  list_of_ids_filtered = list(filter(lambda x: str(x)!='nan',list_of_ids))
  result = {}
  counter = 0
  fails = []
  for x in list_of_ids_filtered:
    if counter%10 == 0:
      print("Currently at ",counter)
    try:
      result.update(request_pdb_id_sequence(x))
    except:
      print("Failed: ",x)
      fails.append(x)
    time.sleep(1)
    counter += 1
  with open(destination_save, 'w') as fout:
    json.dump(result, fout)
  print("Success")
  return fails

In [0]:
failed_requests = create_pdb_sequence_dict_requester(pdb_ids,sequence_dict_loc)



```
Currently at  0
Currently at  10
Currently at  20
Currently at  30
Currently at  40
Success
```



### Create mutations

In [0]:
mutations = T251_table[["MUTATION"]].copy()

In [0]:
mutations.head()

,MUTATION
0,1ADO@A@D128A
1,1ADO@A@D128G
2,1ADO@A@D128N
3,1ADO@A@D128Q
4,1ADO@A@D128V


In [0]:
#retrieve sequence dict
pdb_sequence_dict = None
with open(sequence_dict_loc, 'r') as f:
        pdb_sequence_dict = json.load(f)

In [0]:
def find_corresponding_sequence(protein_id,seq_dict):
  assert len(protein_id.split("_"))==1, "Invalid pdb id."
  if "A" in seq_dict[protein_id]:
    return seq_dict[protein_id]["A"]
  elif "1" in seq_dict[protein_id]:
    return seq_dict[protein_id]["1"]
  elif len(seq_dict[protein_id])==1:
    return seq_dict[protein_id][list(seq_dict[protein_id].keys())[0]]
  else:
    print("WARNING: ",protein_id, " (Took first sequence due to ambiguity!)") #I know it happens only one time, and first seqence matches mutations...
    return seq_dict[protein_id][list(seq_dict[protein_id].keys())[0]]

In [0]:
def mutate_seq(seq_id, seq, mutation, offset_dict):
  assert len(seq_id.split("_"))==1, "Invalid pdb id."
  mut_idx = int(mutation[1:-1])-1
  wt = mutation[0]
  mut = mutation[-1]
  
  #handle offset lists
  if seq_id in offset_dict:
    for offset in offset_dict[seq_id]:
      if mut_idx-offset<len(seq) and mut_idx-offset>=0:
        if seq[mut_idx-offset]==wt:
          mut_idx -= offset
          return seq[:mut_idx]+mut.strip()+seq[mut_idx+1:], offset
  
  if mut_idx >=len(seq) or mut_idx<0:
    print("index error: ", seq_id, mutation)
    
  if seq[mut_idx] != wt:
    print("Mutation mismatch! ID: ", seq_id, " Mutation: ", mutation, " Index: ", mut_idx, " Found: ",seq[mut_idx])
  return seq[:mut_idx]+mut.strip()+seq[mut_idx+1:], 0 

In [0]:
def find_base_sequence(mutation_encoding, seq_dict, start_offset_dict):
  protein_id = mutation_encoding.split("@")[0]
  temp_split = protein_id.split("_") #just take the sequence as is from the dictionary
  if len(temp_split)==1:
    return find_corresponding_sequence(temp_split[0], seq_dict)
  elif len(temp_split[1]) <= 2: #truncated ones: dont really know how they were truncated, just take the default sequence
    return find_corresponding_sequence(temp_split[0], seq_dict)
    print("Truncated: ", protein_id)
  else: #premutate sequences appropriately
    temp_seq, temp_offset = mutate_seq(temp_split[0], find_corresponding_sequence(temp_split[0], seq_dict), temp_split[1], start_offset_dict)
    return temp_seq

In [0]:
#manually looked up offsets for sequences:
start_offset_dict = {"1B5M":[2],"1CLW":[112],"1FTG":[1],"1HFZ":[-1],"1KDX":[585],"1LS4":[-16],"1N0J":[-1],"1TFE":[54],"1YPI":[1]} #every offset as in PDB

In [0]:
mutations["base_sequence"] = mutations["MUTATION"].apply(lambda x: find_base_sequence(x, pdb_sequence_dict, start_offset_dict)) 



```
WARNING:  1ANT  (Took first sequence due to ambiguity!)
WARNING:  1HYN  (Took first sequence due to ambiguity!)
WARNING:  1HYN  (Took first sequence due to ambiguity!)
```



In [0]:
#tmp = "HGSPVDICTAKPRDIPMNPMCIYRSPEKKATEDEGSEQKIPEATNRRVWELSKANSRFATTFYQHLADSKNDNDNIFLSPLSISTAFAMTKLGACNDTLQQLMEVFKFDTISEKTSDQIHFFFAKLNCRLYRKANKSSKLVSANRLFGDKSLTFNETYQDISELVYGAKLQPLDFKENAEQSRAAINKWVSNKTEGRITDVIPSEAINELTVLVLVNTIYFKGLWKSKFSPENTRKELFYKADGESCSASMMYQEGKFRYRRVAEGTQVLELPFKGDDITMVLILPKPEKSLAKVEKELTPEVLQEWLDELEEMMLVVHMPRFRIEDGFSLKEQLQDMGLVDLFSPEKSKLPGIVAEGRDDLYVSDAFHKAFLEVNEEGSEAAASTAVVIAGRSLNPNRVTFKANRPFLVFIREVPLNTIIFMGRVANPCVK"
#1ANT

In [0]:
#temp2 = "MEELQDDYEDMMEENLEQEEYEDPDIPESQMEEPAAHDTEATATDYHTTSHPGTHKVYVELQELVMDEKNQELRWMEAARWVQLEENLGENGAWGRPHLSHLTFWSLLELRRVFTKGTVLLDLQETSLAGVANQLLDRFIFEDQIRPQDREELLRALLLKHSHAGELEALGGVKPAVLTRSGDPSQPLLPQHSSLETQLFCEQGDGGTEGHSPSGILEKIPPDSEATLVLVGRADFLEQPVLGFVRLQEAAELEAVELPVPIRFLFVLLGPEAPHIDYTQLGRAAATLMSERVFRIDAYMAQSRGELLHSLEGFLDCSLVLPPTDAPSEQALLSLVPVQRELLRRRYQSSPAKPDSSFYKGLDLNGGPDDPLQQTGQLF" 
#1hyn

In [0]:
def create_mutations_offset_columns(mutations_df, start_offset_dict):
  temp_list = []
  temp_list2 = []
  for i in range(len(mutations_df)):
    temp_split = mutations_df.iloc[i]["MUTATION"].split("@")
    tmp1, tmp2 = mutate_seq(seq_id=temp_split[0].split("_")[0], seq=mutations_df.iloc[i]["base_sequence"], mutation=temp_split[2], offset_dict=start_offset_dict)
    temp_list.append(tmp1)
    temp_list2.append(tmp2)
  return temp_list, temp_list2

In [0]:
mutated_sequence, offsets = create_mutations_offset_columns(mutations,start_offset_dict)

In [0]:
mutations["mutated_sequence"] = mutated_sequence
mutations["mutation_offsets"] = offsets

In [0]:
mutations.head()

,MUTATION,base_sequence,mutated_sequence,mutation_offsets
0,1ADO@A@D128A,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,0
1,1ADO@A@D128G,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,0
2,1ADO@A@D128N,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,0
3,1ADO@A@D128Q,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,0
4,1ADO@A@D128V,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,PHSHPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSI...,0


In [0]:
#check mutations:
for i in range(len(mutations)):
  mut = mutations.iloc[i]["MUTATION"].split("@")[2]
  index = int(mut[1:-1])-1-mutations.iloc[i]["mutation_offsets"]
  if mutations.iloc[i]["base_sequence"][index] != mut[0] or mutations.iloc[i]["mutated_sequence"][index] != mut[-1]:
    print("Upsi!")
print("Done!")



```
Done!
```



## Merge dataframes and export to CSV

In [0]:
T251_table["base_sequence"] = mutations.base_sequence
T251_table["mutated_sequence"] = mutations.mutated_sequence
T251_table["mutation_offsets"] = mutations.mutation_offsets

In [0]:
T251_table.to_csv(target_data_folder_loc+"T251_with_sequences.csv")

In [0]:
mutations.to_csv(target_data_folder_loc+"T251_sequences_only.csv")

## Transform sequence data to numpy arrays

In [0]:
# check for unusable sequences:
print((T251_table["base_sequence"].apply(lambda x: len(x))>650).sum())
print((T251_table["base_sequence"].apply(lambda x: len(x))<50).sum())



```
0
0
```



In [0]:
IUPAC_Extended_Dic_Transf = {"A":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"K":9,"L":10,"M":11,"N":12,"P":13,"Q":14,"R":15,"S":16,"T":17,"V":18,"W":19,"Y":20,"B":21,"X":21,"Z":21,"J":21,"U":21,"O":21}

In [0]:
def seq_transf_pad(seq, target_len):
    return np.pad(np.array(list(map(lambda x: IUPAC_Extended_Dic_Transf[x], seq)),dtype="int8"),(0, target_len - len(seq)), 'constant')

In [0]:
# only mesophilic xor thermophilic
def get_X_padded_tranf(df_u,column_name,seq_len=650):
    df_u = df_u[[column_name]]
    X_all = np.zeros((len(df_u),seq_len),dtype="int8")

    for i in range(len(df_u)):
        X_all[i] = seq_transf_pad(df_u.iloc[i][column_name],seq_len)

    return X_all

In [0]:
X_wild = get_X_padded_tranf(T251_table, column_name="base_sequence", seq_len=650)

In [0]:
X_mut = get_X_padded_tranf(T251_table, column_name="mutated_sequence", seq_len=650)

In [0]:
# check for unwanted amino acids
print(np.array(list(map(lambda x: 21 in x, X_wild))).sum())
print(np.array(list(map(lambda x: 21 in x, X_mut))).sum())



```
0
0
```



In [0]:
np.save(target_data_folder_loc+"T251_X_wild", X_wild)

In [0]:
np.save(target_data_folder_loc+"T251_X_mut", X_mut)